In [1]:
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Abro los csv
df_flights = pd.read_csv("Customer Flight Activity.csv")
df_loyalty = pd.read_csv("Customer Loyalty History.csv")

## FASE 1: EXPLORACIÓN Y LIMPIEZA ##

In [3]:
# EXPLORACIÓN INICIAL

In [4]:
# Exploro las 5 primeras filas de "df_flights"
df_flights.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [5]:
# Exploro las 5 últimas filas de "df_flights"
df_flights.tail()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
405619,999902,2018,12,0,0,0,0,0.0,0,0
405620,999911,2018,12,0,0,0,0,0.0,0,0
405621,999940,2018,12,3,0,3,1233,123.0,0,0
405622,999982,2018,12,0,0,0,0,0.0,0,0
405623,999986,2018,12,0,0,0,0,0.0,0,0


In [6]:
# Exploro las 10 filas aleatorias de "df_flights"
df_flights.sample(10)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
290537,273020,2018,6,0,0,0,0,0.0,0,0
331752,666222,2018,8,0,0,0,0,0.0,0,0
199648,830818,2017,12,4,3,7,1267,126.0,0,0
370389,394927,2018,6,15,7,22,1716,171.0,520,42
69954,225529,2017,5,0,0,0,0,0.0,0,0
55917,378968,2017,4,2,2,4,972,97.0,0,0
278157,512219,2018,5,0,0,0,0,0.0,0,0
159515,494213,2017,10,12,0,12,1764,176.0,0,0
292176,359575,2018,6,18,10,28,2604,260.0,0,0
293914,452976,2018,6,1,0,1,1829,182.0,0,0


In [7]:
# Compruebo el total de filas y columnas de "df_flights"
df_flights.shape

(405624, 10)

In [8]:
# Exploro las 5 primeras filas de "df_loyalty"
df_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [9]:
# Exploro las 5 últimas filas de "df_loyalty"
df_loyalty.tail()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
16732,823768,Canada,British Columbia,Vancouver,V6E 3Z3,Female,College,NaN,Married,Star,61850.19,Standard,2012,12,NaN,NaN
16733,680886,Canada,Saskatchewan,Regina,S1J 3C5,Female,Bachelor,89210.0,Married,Star,67907.27,Standard,2014,9,NaN,NaN
16734,776187,Canada,British Columbia,Vancouver,V5R 1W3,Male,College,NaN,Single,Star,74228.52,Standard,2014,3,NaN,NaN
16735,906428,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,-57297.0,Married,Star,10018.66,2018 Promotion,2018,4,NaN,NaN
16736,652627,Canada,Manitoba,Winnipeg,R2C 0M5,Female,Bachelor,75049.0,Married,Star,83325.38,Standard,2015,12,2016.0,8.0


In [10]:
# Exploro las 10 filas aleatorias de "df_loyalty"
df_loyalty.sample(10)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
3639,308362,Canada,Nova Scotia,Halifax,B3C 2M8,Female,Bachelor,63567.0,Married,Aurora,15673.14,Standard,2013,1,NaN,NaN
3524,829316,Canada,Quebec,Quebec City,G1B 3L5,Female,Bachelor,50401.0,Single,Aurora,14065.30,Standard,2012,6,NaN,NaN
2650,732524,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,60208.0,Married,Aurora,8586.80,Standard,2015,8,NaN,NaN
2155,877008,Canada,British Columbia,Whistler,V6T 1Y8,Male,Bachelor,19203.0,Married,Nova,2926.83,2018 Promotion,2018,4,NaN,NaN
8321,721300,Canada,Quebec,Montreal,H2T 2J6,Male,Bachelor,70816.0,Single,Nova,8736.50,Standard,2016,11,NaN,NaN
7817,884708,Canada,Ontario,Ottawa,K1F 2R2,Male,Bachelor,73131.0,Married,Nova,7765.23,Standard,2015,9,NaN,NaN
3025,659112,Canada,Alberta,Banff,T4V 1D4,Male,Bachelor,81585.0,Single,Aurora,9942.30,Standard,2016,8,NaN,NaN
4883,499346,Canada,Ontario,Toronto,P1J 8T7,Female,Bachelor,77914.0,Divorced,Nova,3230.58,Standard,2018,6,NaN,NaN
5624,899009,Canada,Quebec,Montreal,H2T 2J6,Female,Bachelor,55476.0,Divorced,Nova,4111.20,Standard,2016,9,NaN,NaN
5981,276806,Canada,New Brunswick,Fredericton,E3B 2H2,Female,Bachelor,82147.0,Single,Nova,6019.93,2018 Promotion,2018,4,NaN,NaN


In [11]:
# Compruebo el total de filas y columnas de "df_loyalty"
df_loyalty.shape

(16737, 16)

In [12]:
# Compruebo valores nulos para "df_flights"
# Observo que en este df NO tengo
df_flights.isnull().sum()

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [13]:
# Compruebo valores nulos para "df_loyalty". 
df_loyalty.isnull().sum()

Loyalty Number            0
Country                   0
Province                  0
City                      0
Postal Code               0
Gender                    0
Education                 0
Salary                 4238
Marital Status            0
Loyalty Card              0
CLV                       0
Enrollment Type           0
Enrollment Year           0
Enrollment Month          0
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

In [14]:
# En "df_loyalty" encuentro que tengo nulos en las columnas: "Salary", "Cancellation Year", "Cancellation Month". Miro de que tipo son estas columnas.
df_loyalty.dtypes

Loyalty Number          int64
Country                object
Province               object
City                   object
Postal Code            object
Gender                 object
Education              object
Salary                float64
Marital Status         object
Loyalty Card           object
CLV                   float64
Enrollment Type        object
Enrollment Year         int64
Enrollment Month        int64
Cancellation Year     float64
Cancellation Month    float64
dtype: object

In [15]:
# Lo observo en procentaje. En el siguiente punto de la fase exploración, analizo como voy a proceder con los datos nulos.
df_loyalty.isnull().sum() / df_loyalty.shape[0] * 100

Loyalty Number         0.000000
Country                0.000000
Province               0.000000
City                   0.000000
Postal Code            0.000000
Gender                 0.000000
Education              0.000000
Salary                25.321145
Marital Status         0.000000
Loyalty Card           0.000000
CLV                    0.000000
Enrollment Type        0.000000
Enrollment Year        0.000000
Enrollment Month       0.000000
Cancellation Year     87.650117
Cancellation Month    87.650117
dtype: float64

In [16]:
# Observo si hay duplicado en "df_flights". SI TENGO
df_flights.duplicated().sum()

1864

In [17]:
# Observo si hay duplicado en "df_loyalty". NO TENGO
df_loyalty.duplicated().sum()

0

In [18]:
# Solo tengo duplicados en "df_flights", analizo cuáles son. "keep=False" Duplicados en todas las columnas
# Observo que se repiten los valores en la columna "Loyalty Number"

df_flights[df_flights.duplicated(keep=False)]
 

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
226,112142,2017,1,0,0,0,0,0.0,0,0
227,112142,2017,1,0,0,0,0,0.0,0,0
477,126100,2017,1,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
405111,971370,2018,12,0,0,0,0,0.0,0,0
405409,988392,2018,12,0,0,0,0,0.0,0,0
405410,988392,2018,12,0,0,0,0,0.0,0,0
405436,989528,2018,12,0,0,0,0,0.0,0,0


In [19]:
# Me di cuenta que colocando el indexcol=0 no me cogia "Loyalty Number" como columna. Lo cambié al leer los csv. 
df_flights.columns

Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')

In [ ]:
# Veo los duplicados en la columna "Loyalty Number" 
df_flights.duplicated(subset = "Loyalty Number").sum()

388887

In [22]:
# Antes de unir los DataFrames debo gestionar los duplicados. 
# Como este atributo "Loyalty Number" representa un identificador único para cada cliente, decido eliminar los duplicados.

# Elimino las filas duplicadas del DataFrame
df_flights.drop_duplicates(inplace=True)

# Restablezco el índice del DataFrame
df_flights.reset_index(drop=True)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
403755,999902,2018,12,0,0,0,0,0.0,0,0
403756,999911,2018,12,0,0,0,0,0.0,0,0
403757,999940,2018,12,3,0,3,1233,123.0,0,0
403758,999982,2018,12,0,0,0,0,0.0,0,0


In [ ]:
# Compruebo que se hayan eliminado los duplicados
df_flights.duplicated().sum()

0

In [26]:
# Ahora uniré los dos DataFrames, basándome en la columna común "Loyalty Number"
df_combinado = df_loyalty.merge(df_flights, on =["Loyalty Number"], how = "left")

RECORDATORIO PARA MÍ: (left join)
Si un Loyalty Number de loyalty tiene una coincidencia en flight, se agregarán las columnas de flight a esa fila.
Si un Loyalty Number de loyalty no tiene una coincidencia en flight, se mantendrá la fila de loyalty tal como está, pero las columnas de flight se llenarán con NaN (valores faltantes).

In [29]:
df_combinado.sample(10)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
235066,840659,Canada,Alberta,Banff,T4V 1D4,Male,Bachelor,63211.0,Divorced,Nova,31221.75,Standard,2018,7,NaN,NaN,2017,6,0,0,0,0,0.0,0,0
304639,314935,Canada,Saskatchewan,Regina,S1J 3C5,Female,Bachelor,71265.0,Married,Star,4150.84,Standard,2013,2,NaN,NaN,2017,9,15,7,22,2420,242.0,0,0
199392,967462,Canada,Alberta,Edmonton,T3G 6Y6,Female,Bachelor,47602.0,Divorced,Nova,8608.16,Standard,2015,6,NaN,NaN,2017,1,0,0,0,0,0.0,0,0
278880,375792,Canada,Ontario,Kingston,M9K 2P4,Male,Bachelor,73135.0,Single,Star,2674.73,Standard,2017,11,NaN,NaN,2018,1,10,0,10,1880,188.0,0,0
248175,556778,Canada,Alberta,Peace River,T9O 2W2,Female,Doctor,269695.0,Married,Star,2387.61,Standard,2013,8,NaN,NaN,2017,3,0,0,0,0,0.0,0,0
37426,409768,Canada,Ontario,Toronto,M8Y 4K8,Male,Bachelor,51573.0,Single,Aurora,6005.21,Standard,2018,8,NaN,NaN,2018,1,0,0,0,0,0.0,0,0
55293,933656,Canada,Ontario,Toronto,M1R 4K3,Male,Master,105487.0,Divorced,Aurora,7805.31,Standard,2013,4,NaN,NaN,2017,2,5,5,10,1880,188.0,0,0
357680,334143,Canada,Manitoba,Winnipeg,R2C 0M5,Female,Bachelor,48265.0,Married,Star,7873.71,Standard,2015,4,NaN,NaN,2017,2,4,3,7,1365,136.0,0,0
317317,774234,Canada,Quebec,Montreal,H2T 9K8,Female,High School or Below,68312.0,Divorced,Star,4869.60,Standard,2016,8,NaN,NaN,2017,9,0,0,0,0,0.0,0,0
232694,170982,Canada,Quebec,Montreal,H2Y 4R4,Male,High School or Below,47587.0,Divorced,Nova,24127.50,Standard,2018,12,NaN,NaN,2017,10,0,0,0,0,0.0,0,0
